In [1]:
#default_exp anime.callbacks

In [2]:
#export
import pandas as pd
from pathlib import Path
import json
from functools import partial
from PIL import Image
import numpy as np
from itertools import chain
from fastai2.torch_basics import *
from fastai2.layers import *
from fastai2.data.all import *
from fastai2.data.block import *
from fastai2.optimizer import *
from fastai2.learner import *
from fastai2.metrics import *
from fastai2.callback.all import *
from fastai2.vision.all import *
from anime.ugatit import *
from fastai2.callback.wandb import WandbCallback
import wandb
from fastai2.fp16_utils import convert_module
from anime.ugatit_networks import ILN,adaILN
from fastai2.callback.fp16 import _copy_state
from torch.autograd import Variable
from anime.dataloading import CycleImage

In [3]:
#export
class GANOptimizer(object):
    def __init__(self, gen_optim,disc_optim,params,**kwargs):
        gen_params,disc_params=params
        #might want to try lower lr for the generator
        self.gen_optim,self.disc_optim=gen_optim(gen_params,**kwargs),disc_optim(disc_params,**kwargs)
        self.optimizing_gen=True
    def __getattr__(self, item):
        opt= self.gen_optim if(self.optimizing_gen) else self.disc_optim
        return getattr(opt, item)
    def zero_grad(self):
        self.gen_optim.zero_grad()
        self.disc_optim.zero_grad()
    def state_dict(self):
        gs = self.gen_optim.state_dict()
        ds = self.disc_optim.state_dict()
        return {'hypers':{
                    'gen':gs['hypers'],
                    'disc':ds['hypers']},
                'state':{
                    'gen':gs['state'],
                    'disc':ds['state']}}
    def load_state_dict(self, sdict):
        self.gen_optim.load_state_dict({'hypers': sdict['param_groups']['gen'],'state':sdict['state']['gen']})
        self.disc_optim.load_state_dict({'param_groups': sdict['param_groups']['disc'],'state':sdict['state']['disc']})
#export
class Switch(Callback):
    toward_end=True
    def begin_epoch(self):
        self.learn.disc_pass=False #TODO too many (not disc_pass), switch to gen_pass?!?
        self.model.optimizing_gen= not self.learn.disc_pass
        if(hasattr(self, 'gan_losses')):
            self.learn.loss_func=self.gan_losses[self.learn.disc_pass]
        self.opt.zero_grad()
    def begin_batch(self):
        if(not hasattr(self, 'gan_losses')):
            self.gan_losses=self.loss_func
            self.learn.loss_func=self.gan_losses[self.learn.disc_pass]
        self.opt.zero_grad()
        clear_grads(self.model.parameters())
        self.learn.loss_func=self.gan_losses[self.learn.disc_pass]
    def after_batch(self):
        if(getattr(self.learn,'overflow',False)):
            self.learn.overflow=False
            if(self.learn.disc_pass):
                self.one_batch(self.iter,self.learn.dls.train.one_batch())
            return
        self._switch()
        
    def after_epoch(self):
        self.loss_func=self.gan_losses
    
    def _switch(self):
        if(self.learn.training): self.learn.disc_pass=not self.learn.disc_pass
        self.model.optimizing_gen= not self.learn.disc_pass
        self.opt.optimizing_gen= not self.learn.disc_pass
        self.opt.zero_grad()
        self.learn.loss_func=self.gan_losses[self.learn.disc_pass]
        if(self.learn.disc_pass):
            self.one_batch(self.iter,(self.learn.xb[0],(CycleImage(torch.Tensor([[[[0]]]]).cuda(),torch.Tensor([[[[0]]]]).cuda()))))
#export
class LinearDecreaseLR(Callback):
    def __init__(self,start_epoch=0,end_epoch=None):
        store_attr(self,"start_epoch,end_epoch")
    def begin_fit(self):
        self.LDmax_lr=self.lr
    def end_fit(self):
        self.lr=self.LDmax_lr
    def begin_batch(self):
        end_epoch = self.end_epoch if(self.end_epoch) else self.n_epoch
        current_iter=(self.epoch+self.start_epoch)*self.n_iter+self.iter
        run_length=end_epoch*self.n_iter
        half_run=run_length // 2
        self.LDlr=self.LDmax_lr
        if current_iter > half_run:
            self.LDlr = (run_length-current_iter)*self.LDmax_lr/(run_length-half_run)
        #elif current_iter < half_run/2:
        #    self.lr = self.max_lr*(current_iter+1)/(half_run/2+1)
        else:
            self.LDlr = self.LDmax_lr
        self.opt.set_hyper('lr',self.LDlr)
#export        
class SameXbYb(Callback):
    def begin_batch(self):
        self.learn.xb=(self.learn.xb[0],to_device(self.learn.xb,torch.device('cuda',index=1))[0])
    def after_pred(self):
        self.learn.yb=to_float(self.learn.xb)
#export
class CachedMemoryDebug(Callback):
    def after_batch(self):
        print(torch.cuda.memory_cached(device=torch.device('cuda',index=0))
              ,torch.cuda.memory_cached(device=torch.device('cuda',index=1)))
#export
class OptimizedOptimizer(Optimizer):
    #Changing order to guarantee order of params in optimizer smallest to largest. 
    #This allows params to be cleared before getting to largest param update. 
    def all_params(self, n=slice(None), with_grad=False, sort_key=lambda p: np.prod(p[0].shape)):
        res = super().all_params(n=n, with_grad=with_grad)
        res.sort(sort_key)
        return res
    def zero_grad(self, clear=False):
        for p,*_ in self.all_params(with_grad=True):
            p.grad.detach_()
            p.grad.zero_()
            if(clear): del p.grad
#export
def zero_grad(p, **kwargs):
    "Register the number of steps done in `state` for `p`"
    p.grad.detach_()
    p.grad.zero_()
    del p.grad
    return {}
#export
def zero_Adam(params, lr, mom=0.9, sqr_mom=0.99, eps=1e-5, wd=0., decouple_wd=True):
    "A `Optimizer` for Adam with `lr`, `mom`, `sqr_mom`, `eps` and `params`"
    cbs = [weight_decay]
    cbs += [partial(average_grad, dampening=True), average_sqr_grad, step_stat, adam_step,zero_grad]
    return OptimizedOptimizer(params, cbs, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)
#export
def clear_grads(params):
    for p in params:
        if(is_listy(p)):
            clear_grads(p)
        elif(p.grad is not None):
            del p.grad

In [4]:
#export
@typedispatch
def wandb_process(x:CycleImage, y:CycleImage, samples, outs):
    "Process `sample` and `out` depending on the type of `x/y`"
    res = []
    for s,o in zip(samples, outs):
        img = TensorImage(s[0].toTensor()[0])
        #db.valid_dl.decode(s)
        res.append(wandb.Image(img, caption='Input data', grouping=3))
        for t, capt in ((o[0], "Prediction"), (s[1], "Ground Truth")):
            # Resize plot to image resolution (from https://stackoverflow.com/a/13714915)
            my_dpi = 100
            fig = plt.figure(frameon=False, dpi=my_dpi)
            h, w = img.shape[:2]
            fig.set_size_inches(w / my_dpi, h / my_dpi)
            ax = plt.Axes(fig, [0., 0., 1., 1.])
            ax.set_axis_off()
            fig.add_axes(ax)
            # Superimpose label or prediction to input image
            ax = img.show(ctx=ax)
            #import pdb; pdb.set_trace()
            ax = t.show(ctx=ax)
            res.append(wandb.Image(fig, caption=capt))
            plt.close(fig)
    return res

In [5]:
#export
class MetricRecorder(Callback):
    def __init__(self):
        super().__init__()
        self.remove_on_fetch=True
    def after_batch(self):
        mets = self.recorder._train_mets if self.training else self.recorder._valid_mets
        self.learn.metric_values=dict([(met.name, met.value) for met in mets])

In [6]:
#export
class Conditional_Loss(AvgSmoothLoss):
    def __init__(self, f=lambda x:True,name='None',beta=0.98,): 
        super(Conditional_Loss,self).__init__(beta)
        self.f=f
        self.metric_name=name
    def accumulate(self, learn):
        if(self.f(learn)):
            super().accumulate(learn)
            
            setattr(learn, self.metric_name, self.value)
    @property
    def name(self):
        return self.metric_name

In [7]:
#https://github.com/NVIDIA/apex/blob/03a25ba8a61d15a503405ef4bbda83724bb531b2/apex/fp16_utils/fp16util.py
# had to edit due to custom normalization layers
#export
def convert_module_alt(module, dtype):
    """
    Converts a module's immediate parameters and buffers to dtype.
    """
    for param in module.parameters(recurse=False):
        if param is not None:
            if param.data.dtype.is_floating_point:
                param.data = param.data.to(dtype=dtype)
            if param._grad is not None and param._grad.data.dtype.is_floating_point:
                param._grad.data = param._grad.data.to(dtype=dtype)
    spectral_norm_buffers=filter(lambda b:b[0] in ('weight_u','weight_v','weight_orig'),module.named_buffers(recurse=False))
    spectral_norm_buffers=list(map(lambda b: b[1],spectral_norm_buffers))
    spectral_norm_buffers=[]
    for buf in module.buffers(recurse=False):
        if buf is not None and buf.data.dtype.is_floating_point and 0==len(list(filter(lambda b:b.shape==buf.shape and torch.all(torch.eq(b, buf)),spectral_norm_buffers))):
            buf.data = buf.data.to(dtype=dtype)
#export
def convert_network(network, dtype):
    """
    Converts a network's parameters and buffers to dtype.
    """
    for module in network.modules():
        if isinstance(module, torch.nn.modules.batchnorm._BatchNorm) and module.affine is True:
            continue
        if isinstance(module, (ILN,adaILN,nn.InstanceNorm2d,nn.LayerNorm)):
            continue
        convert_module_alt(module, dtype)
        if isinstance(module, torch.nn.RNNBase) or isinstance(module, torch.nn.modules.rnn.RNNBase):
            module.flatten_parameters()
    return network

In [8]:
#export
def GAN_splitter(model):
    gen_params=itertools.chain(model.models['GA2B'].parameters(),model.models['GB2A'].parameters())
    disc_params=itertools.chain(model.models['DA'].parameters(),model.models['DB'].parameters(),model.models['LA'].parameters(),model.models['LB'].parameters())
    return (gen_params,disc_params)
#export
class GANWandb(WandbCallback):
    run_before=Switch
    def after_batch(self):
        "Log hyper-parameters and training loss"
        g_scale,d_scale = (self.learn.mixed_precision_tuple.gloss_scale,self.learn.mixed_precision_tuple.dloss_scale) if(hasattr(self.learn,'mixed_precision_tuple')) else (1,1)
        if self.training and not self.learn.disc_pass:
            self._wandb_step += 1
            self._wandb_epoch += 1/self.n_iter
            hypers = {f'{k}G_{i}':v for i,h in enumerate(self.opt.hypers) for k,v in h.items()}
            wandb.log({'epoch': self._wandb_epoch,'loss': self.smooth_loss, **hypers, **self.learn.metric_values, 'loss_scaleG': g_scale}, step=self._wandb_step)
        elif self.training and self.learn.disc_pass:
            hypers = {f'{k}D_{i}':v for i,h in enumerate(self.opt.hypers) for k,v in h.items()}
            wandb.log({'epoch': self._wandb_epoch,'loss': self.smooth_loss, **hypers, **self.learn.metric_values, 'loss_scaleD': d_scale}, step=self._wandb_step)
    #Fixing memory usage
    def after_epoch(self):
        "Log validation loss and custom metrics & log prediction samples"
        # Correct any epoch rounding error and overwrite value
        self._wandb_epoch = round(self._wandb_epoch)
        wandb.log({'epoch': self._wandb_epoch}, step=self._wandb_step)
        # Log sample predictions
        if self.log_preds:
            b = self.valid_dl.one_batch()
            with torch.no_grad():
                self.learn.model.eval()
                self.learn.one_batch(0, b)
                self.learn.model.train()
            preds = getattr(self.loss_func, 'activation', noop)(self.pred)
            out = getattr(self.loss_func, 'decodes', noop)(preds)
            x,y,its,outs = self.valid_dl.show_results(b, out, show=False, max_n=self.n_preds)
            wandb.log({"Prediction Samples": wandb_process(x, y, its, outs)}, step=self._wandb_step)
        wandb.log({n:s for n,s in zip(self.recorder.metric_names, self.recorder.log) if n not in ['train_loss', 'epoch', 'time']}, step=self._wandb_step)

Try using just the generator, discriminator, and pre-trained GAN output. Loss function would be adverserial loss + MSE-ish loss with pre-trained GANs output. LR would have to decrease overtime and/or wight between adv loss and MSE loss would need to be more adverserial. 

In [9]:
#export
def get_master(opt, flat_master=False, ): #device=cuda
    model_params = [[param for param in pg if param.requires_grad] for pg in opt.param_groups]
    if flat_master:
        master_params = []
        for pg in model_params:
            mp = parameters_to_vector([param.data.float() for param in pg])
            mp = nn.Parameter(mp, requires_grad=True)
            if mp.grad is None: mp.grad = mp.new(*mp.size())
            master_params.append([mp])
    else:
        master_params = [[nn.Parameter(param.data.float().detach(), requires_grad=True) for param in pg] for pg in model_params]
    return model_params, master_params

In [10]:
#export
def clear_to_master_grads(model_pgs, master_pgs, flat_master=False):
    for (model_params,master_params) in zip(model_pgs,master_pgs):
        for model, master in zip(model_params, master_params):
            if model.grad is not None:
                if master.grad is None:
                    master.grad = Variable(master.data.new(*master.data.size()))
                master.grad.data.copy_(model.grad.data)
                del model.grad
            else:
                master.grad = None

In [11]:
#export
class ModelToHalfAdaLIN(Callback):
    "Use with MixedPrecision callback (but it needs to run at the very beginning)"
    run_before=TrainEvalCallback
    def begin_batch(self): self.learn.xb = to_half(self.learn.xb)
#export
class MixedPrecisionTuple(MixedPrecision):
    def __init__(self,multi_loss=False,gdiv_factor=2,ddiv_factor=2,**kwargs):
        super().__init__(**kwargs)
        self.multi_loss=multi_loss
        self.gloss_scale=self.loss_scale
        self.dloss_scale=self.loss_scale
        self.gdiv_factor=gdiv_factor
        self.ddiv_factor=ddiv_factor
    def begin_fit(self):
        if self.learn.opt is None: self.learn.create_opt()
        self.gen_model_pgs,self.gen_master_pgs = get_master(self.opt.gen_optim, self.flat_master)
        #Changes the optimizer so that the optimization step is done in FP32.
        _copy_state(self.learn.opt.gen_optim, self.gen_model_pgs, self.gen_master_pgs)
        self.disc_model_pgs,self.disc_master_pgs = get_master(self.opt.disc_optim, self.flat_master)
        #Changes the optimizer so that the optimization step is done in FP32.
        _copy_state(self.learn.opt.disc_optim, self.disc_model_pgs, self.disc_master_pgs)

        if self.dynamic: 
            self.g_count = self.d_count = self.count = 0
    def begin_batch(self):
        if(self.learn.disc_pass):
            self.model_pgs,self.master_pgs= (self.disc_model_pgs,self.disc_master_pgs)
        else: 
            self.model_pgs,self.master_pgs = (self.gen_model_pgs,self.gen_master_pgs)
        if(self.multi_loss): 
            self.loss_scale=self.dloss_scale if self.learn.disc_pass else self.gloss_scale
            self.div_factor=self.ddiv_factor if self.learn.disc_pass else self.gdiv_factor
            self.count=self.d_count if(self.learn.disc_pass) else self.g_count
    def after_pred(self): 
        self.learn.pred = to_float(self.learn.pred)
    def find_overflow(self,ps):
        overflow=[]
        for n,p in ps:
            if p.grad is not None and test_overflow(p.grad.data): overflow.append((n,p.detach().size()))
        return overflow
    def _super_after_backward(self):
        self.learn.loss /= self.loss_scale #To record the real loss
        #First, check for an overflow
        dynamic_loss_overflow = (self.dynamic and (not self.multi_loss) 
                                 and grad_overflow(self.gen_model_pgs+self.disc_model_pgs)) 
        multi_loss_overflow = (self.multi_loss and (((not self.learn.disc_pass) and grad_overflow(self.gen_model_pgs)) 
                               or ( self.learn.disc_pass and grad_overflow(self.disc_model_pgs)))) #have to check both for overflow
        if (dynamic_loss_overflow or multi_loss_overflow):
            self.loss_scale = self.loss_scale/self.div_factor if self.loss_scale/self.div_factor >= 4 else 4
            self.count=0
            print(self.find_overflow(self.model.named_parameters()))
            self.model.zero_grad()
            self._set_gd_count()
            raise CancelBatchException() #skip step and zero_grad
        if(self.learn.disc_pass):
            clear_grads(self.gen_master_pgs)
            clear_grads([self.learn.model.models['GA2B'].parameters(),self.learn.model.models['GB2A'].parameters()])
        else:
            clear_grads(self.disc_master_pgs)
            clear_grads([self.learn.model.models['DA'].parameters(),self.learn.model.models['DB'].parameters(),
                         self.learn.model.models['LA'].parameters(),self.learn.model.models['LB'].parameters()])
        
        clear_to_master_grads(self.model_pgs, self.master_pgs, self.flat_master)
        for master_params in self.master_pgs:
            for param in master_params:
                if param.grad is not None: param.grad.div_(self.loss_scale)
        #Check if it's been long enough without overflow
        if self.clip is not None:
            for group in self.master_pgs: nn.utils.clip_grad_norm_(group, self.clip)
        if self.dynamic:
            self.count += 1
            if self.count == self.scale_wait:
                self.count=0
                self.loss_scale *=self.div_factor
            self._set_gd_count()
            
    def _set_gd_count(self):
        if(self.learn.disc_pass):
            self.d_count=self.count
        else:
            self.g_count=self.count
    def after_backward(self):
        self.learn.overflow=False
        try:
            for y in self.learn.yb:
                del y
            self._super_after_backward()
            self.model.zero_grad()
            clear_grads(self.model.parameters())
        except CancelBatchException as e: 
            self.learn.overflow=True
            self.model.zero_grad()
            clear_grads(self.model.parameters())
            raise e from None
        finally:
            if(self.learn.disc_pass):
                self.dloss_scale=self.loss_scale
            else:
                self.gloss_scale=self.loss_scale
    def after_fit(self):
        _copy_state(self.learn.opt.disc_optim, self.disc_master_pgs, self.disc_model_pgs)
        delattr(self, "disc_master_pgs")
        delattr(self, "disc_model_pgs")
        _copy_state(self.learn.opt.gen_optim, self.gen_master_pgs, self.gen_model_pgs)
        delattr(self, "gen_master_pgs")
        delattr(self, "gen_model_pgs")
        delattr(self, "master_pgs")
        delattr(self, "model_pgs")
#export        
class Perf(Callback):
    towards_end=True
    def after_loss(self):
        del self.learn.xb
        del self.learn.yb 

In [15]:
test_overflow??

In [12]:
#export
class MixedPrecisionTupleNo32(MixedPrecision):
    name='mixed_precision_tuple'
    def __init__(self,multi_loss=False,gdiv_factor=2,ddiv_factor=2,**kwargs):
        super().__init__(**kwargs)
        self.multi_loss=multi_loss
        self.gloss_scale=self.loss_scale
        self.dloss_scale=self.loss_scale
        self.gdiv_factor=gdiv_factor
        self.ddiv_factor=ddiv_factor
    def begin_fit(self):
        if self.learn.opt is None: self.learn.create_opt()
        self.gen_model_pgs = self.opt.gen_optim.param_groups
        self.disc_model_pgs = self.opt.disc_optim.param_groups

        if self.dynamic: 
            self.g_count = self.d_count = self.count = 0
    def begin_batch(self):
        if(self.learn.disc_pass):
            self.model_pgs= self.disc_model_pgs
        else: 
            self.model_pgs = self.gen_model_pgs
        if(self.multi_loss): 
            self.loss_scale=self.dloss_scale if self.learn.disc_pass else self.gloss_scale
            self.div_factor=self.ddiv_factor if self.learn.disc_pass else self.gdiv_factor
            self.count=self.d_count if(self.learn.disc_pass) else self.g_count
    def after_pred(self): 
        self.learn.pred = to_float(self.learn.pred)
    def find_overflow(self,ps):
        overflow=[]
        for n,p in ps:
            if p.grad is not None and test_overflow(p.grad.data): overflow.append((n,p.detach().size()))
        return overflow
    def _super_after_backward(self):
        self.learn.loss /= self.loss_scale #To record the real loss
        #First, check for an overflow
        dynamic_loss_overflow = (self.dynamic and (not self.multi_loss) 
                                 and grad_overflow(self.gen_model_pgs+self.disc_model_pgs)) 
        multi_loss_overflow = (self.multi_loss and (((not self.learn.disc_pass) and grad_overflow(self.gen_model_pgs)) 
                               or ( self.learn.disc_pass and grad_overflow(self.disc_model_pgs)))) #have to check both for overflow
        if (dynamic_loss_overflow or multi_loss_overflow):
            self.loss_scale = self.loss_scale/self.div_factor if self.loss_scale/self.div_factor >= 4 else 4
            self.count=0
            print(self.find_overflow(self.model.named_parameters()))
            self.model.zero_grad()
            self._set_gd_count()
            raise CancelBatchException() #skip step and zero_grad
        
        for params in self.model_pgs:
            for param in params:
                if param.grad is not None: param.grad.div_(self.loss_scale)
        #Check if it's been long enough without overflow
        if self.clip is not None:
            for group in self.model_pgs: nn.utils.clip_grad_norm_(group, self.clip)
        if self.dynamic:
            self.count += 1
            if self.count == self.scale_wait:
                self.count=0
                self.loss_scale *=self.div_factor
            self._set_gd_count()
            
    def _set_gd_count(self):
        if(self.learn.disc_pass):
            self.d_count=self.count
        else:
            self.g_count=self.count
    def after_backward(self):
        self.learn.overflow=False
        try:
            for y in self.learn.yb:
                del y
            self._super_after_backward()
            #self.model.zero_grad()
        except CancelBatchException as e: 
            self.learn.overflow=True
            self.model.zero_grad()
            raise e from None
        finally:
            if(self.learn.disc_pass):
                self.dloss_scale=self.loss_scale
            else:
                self.gloss_scale=self.loss_scale
    def after_step(self):
        self.model.zero_grad()
    def after_fit(self):
        delattr(self, "disc_model_pgs")
        delattr(self, "gen_model_pgs")
        delattr(self, "model_pgs")

In [13]:
#export
class SaveModelCallback(Callback):
    "A `TrackerCallback` that saves the model's best during training and loads it at the end."
    def __init__(self, fname='model', with_opt=True):
        super().__init__()
        store_attr(self, 'fname,with_opt')

    def _save(self, name):
        if(self.learn.epoch%20==0):
            self.learn.save(name, with_opt=self.with_opt)

    def after_epoch(self):
        self._save(f'{self.fname}_{self.epoch}')

In [1]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 2020-05-11-FP16-Blog_Post-2.ipynb.
Converted 2020-05-12-GAN-Blog-Post.ipynb.
Converted 2020-06-11-FP16-Copy1.ipynb.
Converted 2020-06-11-FP16.ipynb.
Converted 2020-06-11-Training-GANs-Copy1.ipynb.
Converted 2020-06-11-Training-GANs.ipynb.
Converted 2020-06-11-UGATIT-a-GAN-in-fp16.ipynb.
Converted 5_17_Tutor.ipynb.
Converted Adam Uses too much gpu memory.ipynb.
Converted Callbacks.ipynb.
Converted Dataloading Original.ipynb.
Converted Dataloading-Copy1.ipynb.
Converted Dataloading-Copy2.ipynb.
Converted Dataloading-Copy3.ipynb.
Converted Dataloading-Copy4.ipynb.
Converted Dataloading-Dec31.ipynb.
Converted Dataloading-Oct22.ipynb.
Converted Dataloading-Split.ipynb.
Converted Dataloading.ipynb.
Converted Dataloading_10_14_2019.ipynb.
Converted Kernel Inception Distance.ipynb.
Converted Techniques for using small GPUs.ipynb.
Converted TensorboardCallback.ipynb.
Converted UGATIT-Dec31Backup.ipynb.
Converted UGATIT-networks.ipynb.
Converted UGATIT.ipynb.
Converted UGATIT_Original.